Video presentation is at 
https://drive.google.com/file/d/1nsvGHqaYam5apdTR2A27m1NrzxvT8aTy/view?usp=sharing

***


# TELECOM CUSTOMER CHURN PREDICTION
Team ELAADO is Aabiya Monsoor, Elena Boiko, and Don Krapohl

## Video presentation
https://drive.google.com/file/d/1nsvGHqaYam5apdTR2A27m1NrzxvT8aTy/view?usp=sharing

## Problem Definition
The primary task of this project is to predict customer churn for a telecommunications company. Customer churn refers to the phenomenon where customers discontinue their subscription to a service or switch to another provider. The dataset used for this project includes demographic, service-related, and billing information for customers. Features such as contract type, monthly charges, payment methods, and tenure are crucial in identifying customers who are at risk of leaving the service.

The telecommunications industry is highly competitive, with an annual churn rate of 15-25%. Reducing churn is critical because retaining existing customers is more cost-effective than acquiring new ones. By accurately predicting churn, businesses can implement targeted retention strategies, reducing attrition and improving profitability.

## Project Goal
The goal of this project is to develop a machine learning model that can accurately predict whether a customer will churn. 

**The key objectives are:**

1) Achieve a high level of classification performance, targeting a **ROC-AUC** score of at least 80%.

2) Provide actionable insights into factors contributing to churn to assist in retention strategies.

## Approach
To meet the project objectives, the following steps were undertaken:

### 1. Data Preprocessing:

Addressed missing or invalid values in the total_charges column by imputing them with the mean and converting the column to numeric.
Encoded categorical variables using LabelEncoder and one-hot encoding as appropriate.
Scaled numerical features using StandardScaler and MinMaxScaler to ensure compatibility with machine learning algorithms.
Split the dataset into training and test sets with stratification to preserve the class distribution.

### 2. Feature Engineering:

Performed feature importance analysis using Random Forest to identify key predictors of churn.
Explored dimensionality reduction using Principal Component Analysis (PCA) to evaluate its impact on model performance.

### 3. Model Development:

**Implemented and tuned various machine learning models:**

Logistic Regression

Support Vector Machines (SVM)

Decision Trees

Random Forest

K-Nearest Neighbors (KNN)

Ensemble methods (Voting Classifier, Bagging, XGBoost)

Applied SMOTE to address class imbalance and improve the minority class representation.
Conducted hyperparameter tuning using RandomizedSearchCV to optimize model performance.

### 4. Model Evaluation:

Evaluated models using metrics such as accuracy, F1-score, precision, recall, and ROC-AUC.
Selected the best-performing model based on test ROC-AUC and interpretability.
Compared advanced models with baseline Logistic Regression to determine the most effective approach.

## Motivation
Customer churn poses a significant challenge for telecom companies, directly impacting revenue and market position. By developing a predictive churn model, this project aims to:

Help businesses identify customers at high risk of leaving.
Enable targeted retention strategies, focusing resources on high-value customers.
Provide actionable insights into the drivers of churn, facilitating data-driven decision-making.
Reducing churn not only preserves a company’s customer base but also improves profitability by lowering acquisition costs and increasing lifetime value. A robust churn prediction model equips telecom companies to thrive in a competitive market and maintain customer loyalty effectively.

## Import Libraries

In [ ]:
# basic dataframe and operations
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# manipulation and preprocessing
from sklearn.preprocessing import Normalizer, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

# models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb

# measuring results
from sklearn.metrics import accuracy_score, f1_score, precision_recall_curve,confusion_matrix, mean_absolute_error, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

# warning suppression
import warnings
from sklearn.exceptions import ConvergenceWarning

## Exploratory Data Analysis (EDA)

In this section, we performed EDA to understand the dataset, check for missing values, visualize data distributions, and analyze correlations between features.

### 1. Loading and Inspecting the Dataset

In [ ]:
# Import the csv training dataset to a pandas dataframe
data_raw_input = pd.read_csv('/kaggle/input/customer-churn-prediction-fall-2024/train.csv')

# Show the shape of the dataset
data_raw_input.shape

The dataset contains 5634 rows (observations) and 21 columns (features), including the target variable (Churn).

### 2. Displaying a Sample of the Training Data

To better understand the dataset, we displayed the first 20 rows, including all 21 columns, to examine the data structure and contents.

In [ ]:
# Display a few rows from the training data
data_raw_input.head(20)

**Dataset Overview:**

The dataset consists of 21 columns, including features such as gender, senior_citizen, tenure, and monthly_charges, as well as the target variable label (indicating customer churn).

**Row Example:** Each row represents a customer, with attributes such as demographics, subscription details, and payment information.

**Target Variable (label):**
0: Customer did not churn.
1: Customer churned.

### 3. Checking for Missing Values, Duplicates, and Data Issues

In this step, we checked for null values, missing data (NaNs), and duplicate rows to ensure data integrity. Additionally, we identified columns with invalid values such as strings with spaces in numeric fields.

In [ ]:
# Get the count of nulls per column
# Turns out we don't have any
print("Nulls:")
print(data_raw_input.isnull().sum().sum())
print("Na count:")
print(data_raw_input.isna().sum().sum())
print("Duplicate rows:")
print(data_raw_input.duplicated(keep='first').sum())

# In attempting to change total_charges from string to numeric I received an error that there were some
#   values in there that were just ' ' -- a non-empty string that contains just a space. Now I'll formally detect it.
print(data_raw_input.columns[data_raw_input.isin([' ']).any()])

**Results:**

1) Null Values: No null values were found in the dataset (Nulls: 0).

2) NaNs: Similarly, no missing values (NaNs) were detected (Na count: 0).

3) Duplicates: The dataset has no duplicate rows (Duplicate rows: 0).

4) Invalid Values: 
The column **total_charges** contains invalid entries: strings with spaces (' ') instead of numeric values.

## 2. Data Preprocessing
In this section, we addressed data preprocessing tasks to prepare the dataset for machine learning models and ensured reusability of the preprocessing pipeline for future predictions.

### 2.1 Dealing with total_charges
The column **total_charges** contained invalid entries (empty strings with spaces ' ') and needed to be converted to a numeric data type for analysis and modeling. So, the column needs to be changed to float. We'll first run an imputer over them to replace that, then we'll change to numeric data type.

#### Summary of Results:
**1. Preprocessing of total_charges:**

Successfully handled invalid values, imputed missing entries, and converted the column to numeric.
Ensured that preprocessing could be applied to both training and test data for consistency.

**2. Reusable Pipeline:**

Defined reusable, modular functions for hyperparameter tuning, feature importance visualization, and test set predictions.

Simplified the workflow for future model evaluation and submission.

In [ ]:
# Class: Multiprep
# Purpose: To preprocess all features. 
class MultiPrep:
    feature_encoders={}
    def destroy_encoders(self):
        self.feature_encoders = {}                   # overwrite the old encoders with an empty collection
        
    def fit_text_encoders(self, df_features):
        non_numeric_columns = df_features.select_dtypes(exclude='number').columns   # get a list of non-numeric cols
        for col in non_numeric_columns:         # loop through those columns to fit an encoder
            encoder = LabelEncoder()            # make a new label encoder for this column
            encoder.fit(df_features[col])       # fit this encoder
            self.feature_encoders[col] = encoder     # add it to the encoders collection for later use

    # fit a normalizer for all numerics
    def fit_scaler(self, df_features):
        # note I'm not caching the "this model has not been fit" error, like I would for a real app
        numeric_columns = df_features.select_dtypes(include='number').columns   # get a list of numeric cols
        scaler = StandardScaler()
        scaler.fit(df_features[numeric_columns])    # apply the transform
        self.feature_encoders['numerics'] = scaler  # preserve the numeric scaler 
     
    # encode all labels
    def transform_encode_all(self, df_features):
        # note I'm not caching the "this model has not been fit" error, like I would for a real app
        non_numeric_columns = df_features.select_dtypes(exclude='number')   # get a list of non-numeric cols
        for col in non_numeric_columns:         # loop through them
            df_features[col] = self.feature_encoders[col].transform(df_features[col])    # apply the transform
        return df_features     
    
    # normalize all numerics
    def transform_scale_all(self, df_features):
        # note I'm not caching the "this model has not been fit" error, like I would for a real app
        numeric_columns = df_features.select_dtypes(include='number').columns   # get a list of numeric cols
        df_features[numeric_columns] = self.feature_encoders['numerics'].transform(df_features[numeric_columns])    # apply the transform
        return df_features
            
    def decode_all(self, df_features):
        non_numeric_columns = df_features.select_dtypes(exclude='number')   # get the non-numeric cols
        for col in non_numeric_columns:         # loop through them
            df_features[col] = self.feature_encoders[col].inverse_transform(df_features[col])    # decode the data
    
    
# split the x and y data. Doing it outside the transformation as we don't want to transform the validation
#   set yet
def split_x_y(data_raw):
    # remove the result column from the input parameters
    # also remove the ID column. It carries no signal.
    X_no_label = data_raw.drop('label', axis=1).drop('id', axis=1)

    # Assign class labels for the input data
    y_labels = data_raw['label']      # assign the labels we'll encode in the next block
    return X_no_label, y_labels


# method: transform_features
# purpose: to clean and encode the features of a dataset
# parameters: X_without_label - the raw features without label or id columns
#       transform_only - True or False. If it's the training set we fit and transform otherwise transform only
# returns: X_scaled - cleaned features that are label encoded and scaled
# steps:
#  Drop the label and id from the x
#  Put the label into y
#  replace spaces in total_charges with Nan
#  Recast total_charges to float
#  impute the missing values in total_charges
#  encode the string data
#  scale the numeric data
#  return the x data
def transform_features(X_without_label, transform_only):
   
    # --------------- total_charges processing --------------------
    # change spaces in total_charges to Nan then recast
    X_without_label['total_charges'] = X_without_label['total_charges'].replace(' ', np.nan)
    X_without_label['total_charges'] = pd.to_numeric(X_without_label['total_charges']) # recast as floating point
    print(X_without_label['total_charges'])
    # Now we're missing values so let's impute them
    imputer = SimpleImputer(strategy='mean')
    X_without_label['total_charges'] = imputer.fit_transform(X_without_label['total_charges'].values.reshape(-1,1))
    
    # --------------- end total_charges processing
      
     
    # Encode the string columns and scale the numerics
    if not transform_only:                           # only fit the model if it's the x training set, not validate or predict
        multiprep.fit_text_encoders(X_without_label)                    # fit the encoders
        multiprep.fit_scaler(X_without_label)                      # fit the normalizer
      
    X_without_label = multiprep.transform_scale_all(X_without_label) # normalize all of the numeric columns 
    X_without_label = multiprep.transform_encode_all(X_without_label) # encode all of the text columns 
    
        
    return X_without_label

# method: tune_model
# purpose: to use cross-validation to find the best HPs, fit a model, and do basic scoring on it
# parameters:
#   model - a defined but not already-fit model to search for the best hyperparameters
#   X_from_train - the features from the training set. Calling it this because I don't know which processed version I'll use.
#   y_from_train - the classes for each sample in the training set
# returns:
#   model - the trained model using the "best" found hyperparameters
def tune_model(model, param_grid, X_from_train, y_from_train, scoring='roc_auc'):
    rscv = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        scoring=scoring,
        cv=5,
        random_state=17,
        refit=True)
    rscv = rscv.fit(X_from_train, y_from_train)  # xtrainsig was 0.81, train balanced .834, trainwolabel 0.767
    print(rscv.best_score_)
    print(rscv.best_params_)

    model.fit(X_from_train, y_from_train)
    return model

# method: plot_importances
# purpose: shows class labels and graphs their contribution to the training variance in descending order (scale 0-1.0)
# parameters:
#   X_from_train - the features from the training set
#   features_labels - the column labels for those features in plaintext
#   importances - the relative importance of each feature (scaled 0-1.0)
# returns:
#   none. Print only.
def plot_importances(X_from_train, feature_labels, importances):
    # graphing of most important features from chapter 6 class notes
    indices = np.argsort(importances)[::-1]
    for f in range(X_from_train.shape[1]):
        print("%2d) %-*s %f" % (f + 1, 30,feature_labels[indices[f]],importances[indices[f]]))

    plt.title('Feature importance')
    plt.bar(range(X_from_train.shape[1]),
        importances[indices],
        align='center')
    plt.xticks(range(X_from_train.shape[1]),
        feature_labels[indices], rotation=90)
    plt.xlim([-1, X_from_train.shape[1]])
    plt.tight_layout()
    plt.show()
def write_predictions(model, test_input_path='/kaggle/input/customer-churn-prediction-fall-2024/test.csv', 
                      predict_out_path='submission.csv'):
    # Do predictions on the submission test set and save the output as csv
    data_test_input = pd.read_csv(test_input_path) # get the test inputs

    df_output = pd.DataFrame()
    # remove the ID column and the save it in output_ids
    df_output['id'] = data_test_input['id']                 # set the IDs we'll output but don't predict on them
    data_test_input = data_test_input.drop('id', axis=1)    # drop the id columns

    # Need to encode the test data
    # This uses the tranformers we trained earlier as feature_encoder
    X_test_for_out = transform_features(data_test_input, transform_only=True)

    # Get the probabilities of class 1 (will churn)
    prob_for_samples = model.predict_proba(X_test_for_out)  # get the predictions of both classes
    df_output['label'] = prob_for_samples[:,1]              # write the label column as the predictions of class 1

    df_output.to_csv(predict_out_path, index=False) # write the csv
    print("Predictions written")                            # print a message
    
label_encoders={}
multiprep = MultiPrep()

### 2.2 Identifying Unique Values in Non-Numeric Columns
In this step, we examined all non-numeric columns in the dataset to identify categorical features and their unique values. This helps determine the appropriate preprocessing steps, such as encoding categorical variables and re-typing numeric columns stored as strings.

In [ ]:
# For the label column in the training set
# Show the unique values of the training labels
col_list = data_raw_input.columns.to_list()

for col in data_raw_input.columns:
    if not pd.api.types.is_numeric_dtype(data_raw_input[col]):
        print("{}: {}".format(col, data_raw_input[col].unique()))

#### Observations:
**Categorical Features:**
Columns like **gender, partner, dependents**, and **payment_method** are categorical and need encoding (e.g., one-hot or label encoding).

**Non-Numeric Column for Numeric Data:**
The column **total_charges** contains numeric data stored as strings. Before any encoding, this column must:
Be converted from string to numeric using pd.to_numeric() after handling invalid entries (' ').

**Ordinal Features:**
No clear ordinal features are identified in the dataset. All categorical features appear nominal (no inherent order).

**Special Cases:**
Features like **multiple_lines, internet_service**, and **streaming_tv** include specific categories like 'No phone service' and 'No internet service', which may require careful encoding to preserve meaningful distinctions.

## 3. Visualisation
### 3.1 Test for imbalance

We can see that the classes are imbalanced. We'll generate more samples after the train/test split for the minority class.

In [ ]:
# Get counts for each class to see how imbalanced the classes are if at all
plt.figure(figsize=(8, 6))
plt.title('Class Imbalance Check', fontsize=16)

# Calculate normalized percentages and prepare the DataFrame for visualization
labels = data_raw_input['label'].value_counts(normalize=True).rename_axis('label').reset_index(name='Percentage')

# Create the bar plot
bar_plot = sns.barplot(x='label', y='Percentage', data=labels, palette='pastel')

# Annotate the bars with percentage values
for p in bar_plot.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    bar_plot.annotate(f'{height:.0%}', (x + width/2, y + height*1.02), ha='center', fontweight='bold')

# Add labels
plt.xlabel('Churn (0 = No, 1 = Yes)', fontsize=12)
plt.ylabel('Percentage', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()

# Show the plot
plt.show()

# Print raw counts for reference
counts = data_raw_input['label'].value_counts()
print(counts)


The **dataset is imbalanced**, with a significant skew towards Class 0 (No Churn). 

**This imbalance can:** 
Lead to bias in machine learning models, where the model may prioritize predicting the majority class (0) while underperforming on the minority class (1).
Negatively impact performance metrics such as recall and F1-score for Class 1.

### 3.2 Correlation Heatmap for numerical features

In [ ]:
# Plot correlation heatmap

# Filter numerical columns only (excluding 'id')
numerical_columns = data_raw_input.select_dtypes(include='number').drop(columns=['id']).columns

# Calculate the correlation matrix for numerical features
numerical_correlation = data_raw_input[numerical_columns].corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(numerical_correlation, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap for Numerical Features')
plt.show()


None of the correlations are very strong (>0.5), indicating that these features alone may not be strong predictors of the label. Other features or engineered variables might be needed to improve predictions.

### 3.3 Churn vs. Numeric Features
These plots visualize the distribution of key numeric features (tenure, monthly_charges, total_charges) against churn to understand their relationship with customer retention, as these features represent customer engagement, pricing, and overall spending, which are critical factors influencing churn behavior.

In [ ]:
# Suppress warnings from pandas and seaborn
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # Ignore FutureWarnings
pd.options.mode.use_inf_as_na = True  # Treat inf values as NaN

# Define numeric features to plot
numeric_features = ['tenure', 'monthly_charges', 'total_charges']

# Plot distributions for each numeric feature against churn
for feature in numeric_features:
    plt.figure(figsize=(10, 6))
    sns.histplot(
        data=data_raw_input,
        x=feature,
        hue='label',
        kde=True,
        palette='viridis',
        bins=30,
        element="step"  # Cleaner plot style
    )
    plt.title(f'Distribution of {feature} by Churn')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.show()

#### Conclusion from Tenure, Monthly Charges, and Total Charges Analysis
**1. Tenure Insights:**

High churn rate for new customers: Customers with low tenure are much more likely to churn, suggesting that newer customers are less engaged or satisfied.
Loyalty increases with tenure: Customers with high tenure are significantly less likely to churn, indicating that long-term customers tend to stay loyal.

**2. Monthly Charges Insights:**

Higher churn rate with higher charges: Customers paying higher monthly charges are more likely to churn, potentially due to dissatisfaction with pricing or perceived value.

**3. Total Charges Insights:**

Loyalty linked to cumulative spending: Customers with higher total charges (long-term, high-value customers) appear more loyal, reflecting their prolonged engagement and satisfaction.
Churn risk for new customers: Lower total charges are associated with higher churn, indicating that newer or less-engaged customers are at greater risk.

### 3.4 Categorical Features and Churn

In [ ]:
# Bar plot for categorical features
categorical_features = ['contract', 'payment_method', 'online_security', 'gender', 
                        'paperless_billing', 'tech_support', 'multiple_lines']

for feature in categorical_features:
    plt.figure(figsize=(10, 6))
    churn_percentage = data_raw_input.groupby(feature)['label'].mean().reset_index()
    sns.barplot(data=churn_percentage, x=feature, y='label', palette='pastel')
    plt.title(f'Churn Rate by {feature}')
    plt.xlabel(feature)
    plt.ylabel('Churn Rate')
    plt.show()

### 3.5 Visualizing Churn Distribution for Categorical Features
We used these plots to visualize the churn distribution across categorical features, enabling us to identify patterns or relationships between customer attributes and churn behavior for more informed feature selection and analysis.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Filter object-type columns, excluding the target variable ('label')
categorical_features = [col for col in data_raw_input.select_dtypes(include='object').columns if col != 'label']

# Dynamically determine the number of rows and columns based on the number of categorical features
num_features = len(categorical_features)
num_rows = (num_features // 4) + (1 if num_features % 4 != 0 else 0)  # Calculate rows needed

# Set up the figure and axes for subplots
fig, axes = plt.subplots(num_rows, 4, figsize=(20, 5 * num_rows))  # Adjust height dynamically
axes = axes.flatten()  # Flatten to make indexing easier

# Generate bar plots for each categorical feature
for i, col in enumerate(categorical_features):
    churn_percentage = data_raw_input.groupby(col)['label'].mean().reset_index()
    sns.barplot(data=churn_percentage, x=col, y='label', palette='pastel', ax=axes[i])
    axes[i].set_title(f'{col} vs Churn')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Churn Rate')

# Remove any unused subplot axes
for j in range(len(categorical_features), len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()
plt.show()


## 4. Train/Test Split and Preprocessing
1) Split the dataset into 70% training data and 30% testing data using a stratified split to ensure consistent class distribution across subsets.
   
2) Applied transformations to the features, such as encoding categorical variables and scaling numerical ones, using a reusable pipeline (transform_features).

### 4.1 Train/Test Split 

In [ ]:
# Split the features and the class labels
X_without_label, y_values = split_x_y(data_raw_input)   # Separate features (X) and target variable (y)

# Perform a 70/30 train-test split
# Stratification ensures class distribution remains consistent across train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_without_label, y_values,
    test_size=0.3,
    stratify=y_values, 
    random_state=17)

# Apply preprocessing (e.g., encoding, scaling) to training and testing sets
# The senior_citizen column is already encoded as 0/1; scaling will retain its interpretability

X_train= transform_features(X_train, transform_only=False) # Fit and transform on training data
X_test = transform_features(X_test, transform_only=True) # Fit and transform on testing data

### 4.2 Transforming senior_citizen Separately

Check how the senior_citizen column appears after preprocessing and transformation.
senior_citizen is already encoded as 0 (not a senior) or 1 (senior), so its transformation should preserve this representation.

Display the first 25 rows of the training and testing sets after applying transformations to verify correctness.

In [ ]:
# Display the first 25 rows of training and testing sets to examine transformations
print(X_train[:25]) # First 25 rows of the training set
print(X_test[:25]) # First 25 rows of the testing set

**Verification:** Ensures transformations are applied correctly without introducing unexpected issues.

**Validation:** Confirms that preprocessing preserves feature integrity (e.g., senior_citizen still conveys the same meaning after scaling).

### 4.3 Balancing the Dataset: Generating Synthetic Samples for the Minority Class
In this step, we address the class imbalance observed in the training dataset. The minority class is significantly smaller (around one-third the size of the majority class). To ensure the model is not biased towards the majority class, we use **Synthetic Minority Oversampling Technique (SMOTE)** to generate synthetic samples for the minority class, bringing the two classes into balance.

In [ ]:
# Generate synthetic samples for the minority class using SMOTE
smote = SMOTE(random_state=17)
X_train_sampled, y_train_sampled = smote.fit_resample(X_train, y_train)

from collections import Counter
print("Original class distribution:", Counter(y_train))
print("Resampled class distribution:", Counter(y_train_sampled))

#### Description of Results:
**Original Class Distribution:**

The training set had a class imbalance, with the minority class comprising roughly one-third of the total samples.

**After SMOTE Sampling:**

The fit_resample method generated synthetic samples for the minority class until the classes were balanced.
Both the majority and minority classes now have the same number of samples.

### 4.4 Feature Importance Ranking

This step evaluates the **importance of features** in the dataset by calculating their relative contribution to the model's decision-making process. Understanding feature importance helps identify the most influential predictors and can guide feature selection or dimensionality reduction.

In [ ]:
# Exclude 'id' from feature labels
feature_labels = data_raw_input.drop(columns=['id']).columns  # Remove 'id' column

# Train the Random Forest Classifier
rf_manual = RandomForestClassifier(max_features=40, n_estimators=200, criterion='gini', n_jobs=-1, random_state=17)
rf_manual.fit(X_train, y_train)  # Fit the model

# Get feature importances
importances = rf_manual.feature_importances_

# Plot the feature importances
plot_importances(X_train, feature_labels, importances)  # Graph the importance of each feature


**Top Features:**

payment_method, monthly_charges, and streaming_movies are the most important predictors, significantly influencing the model's performance.
These features likely correlate strongly with customer churn.

**Less Important Features:**

Features such as tenure, multiple_lines, and tech_support have minimal contribution to the model's decision-making process.


## 5. Model Initialization, Training, and Evaluation 
**Model Evaluation:**
Predictions are made on the test set (X_test), and the model's performance is evaluated using: 

**Accuracy Score:** Percentage of correct predictions.

**F1-Score:** Balance between precision and recall, especially useful for imbalanced datasets.

**ROC-AUC Score:** Evaluates the model's ability to distinguish between classes.
### 5.1 Logistic Regression 
### 5.1.1 Baseline Model with Hyperparameter Tuning
The goal of this step is to train a Logistic Regression (LR) model on the entire feature set and evaluate its performance. The model is optimized by tuning key hyperparameters (solver and C) using RandomizedSearchCV. This serves as a baseline for comparison with future models and dimensionality reduction techniques.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve

# Define the parameter grid for tuning
param_grid = {'solver': ['liblinear', 'newton-cg', 'saga', 'lbfgs'], 
              'C': [0.001, 0.01, 1.0, 3.0, 5.0, 7.0, 10.0, 12.0],
             }

# Tune the hyperparameters "solver" and "C" for the LR model using RandomizedSearchCV
lr_baseline = tune_model(LogisticRegression(random_state=17, max_iter=1000), 
                         param_grid, 
                         X_train, 
                         y_train, 
                         scoring='roc_auc')

# Compute predicted probabilities for Logistic Regression Baseline
y_test_proba_lr_baseline = lr_baseline.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_pred = lr_baseline.predict(X_test)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_pred)  # Accuracy
f1 = f1_score(y_test, y_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_lr_baseline)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_test_proba_lr_baseline)  # For ROC curve plotting later


#### Analysis of Logistic Regression (Baseline):

The accuracy of 80% is reasonable but not exceptional, given the imbalance in the dataset.
The F1-Score of ~60% indicates that the model struggles with precision and recall on the minority class.
The AUC-ROC score of 83.83% demonstrates good discriminatory power, meaning the model is reasonably effective at distinguishing between the positive and negative classes. While better than a random guess (AUC = 0.50), there is still room for improvement in separating the classes more confidently.


### 5.1.2 PCA and Logistic Regression: Dimensionality Reduction and Evaluation
This step evaluates the effect of Principal Component Analysis (PCA) as a dimensionality reduction technique on the performance of a Logistic Regression model. PCA reduces the feature space to 6 components, aiming to simplify the model while retaining most of the variance in the dataset.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve
from sklearn.decomposition import PCA

# Apply PCA for dimensionality reduction
pca = PCA(n_components=6)                   # Reduce to 6 components based on prior analysis
X_pca = pca.fit_transform(X_train)          # Fit and transform the training data

# Tune Logistic Regression on the PCA-transformed data
param_grid = {'solver': ['liblinear', 'newton-cg', 'saga', 'lbfgs'], 
              'C': [0.001, 0.01, 1.0, 3.0, 5.0, 7.0, 10.0, 12.0]}

# Tune the hyperparameters "solver" and "C" for the LR model using RandomizedSearchCV cross-validation
lr_pca = tune_model(LogisticRegression(random_state=17, max_iter=1000), 
                    param_grid, 
                    X_pca, 
                    y_train, 
                    scoring='roc_auc')

# Transform test data using the same PCA model
X_test_pca = pca.transform(X_test)

# Compute predicted probabilities for Logistic Regression on PCA-transformed data
y_test_proba_lr_pca = lr_pca.predict_proba(X_test_pca)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_pred = lr_pca.predict(X_test_pca)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_pred)  # Accuracy
f1 = f1_score(y_test, y_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_lr_pca)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_lr_pca, tpr_lr_pca, _ = roc_curve(y_test, y_test_proba_lr_pca)  # For ROC curve plotting later


#### Summary of PCA Impact:
**Dimensionality Reduction:**
PCA reduced the feature set to 6 components, simplifying the model but causing information loss, which lowered test performance.

**Baseline vs. PCA Performance:**
Baseline Accuracy: 80.01% (without PCA).
PCA Accuracy: 78.30%.
Retaining the full feature set appears more effective for this dataset.

**Impact on F1-Score and AUC:**
PCA reduced both F1-Score (57.11%) and AUC (82.22%) compared to the baseline, indicating weaker performance in balancing class predictions.

### 5.1.3 Feature Selection with Logistic Regression
This step explores whether feature selection improves model performance by using a threshold-based method (SelectFromModel) to remove less important features before training a Logistic Regression model.

In [ ]:
from sklearn.feature_selection import SelectFromModel
import warnings
warnings.filterwarnings("ignore")  # Suppress warnings for cleaner output

# Train Logistic Regression on the full feature set
lr_sel = LogisticRegression(random_state=17, max_iter=1000).fit(X_train, y_train)

# Select important features using SelectFromModel
sfm = SelectFromModel(estimator=lr_sel, prefit=True, threshold=0.05)  # Select features based on importance
X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)

# Tune hyperparameters on the selected feature set
lr_sel = tune_model(
    LogisticRegression(random_state=17, max_iter=1000),
    param_grid, 
    X_train_selected, 
    y_train, 
    scoring='roc_auc'
)

# Compute predicted probabilities for Logistic Regression on the selected feature set
y_test_proba_lr_sel = lr_sel.predict_proba(X_test_selected)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_pred = lr_sel.predict(X_test_selected)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_pred)  # Accuracy
f1 = f1_score(y_test, y_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_lr_sel)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_lr_sel, tpr_lr_sel, _ = roc_curve(y_test, y_test_proba_lr_sel)  # For ROC curve plotting later


#### Feature Selection Impact:

**Feature selection did not improve performance:**
Accuracy (79.66%), F1-Score (59.34%), and AUC-ROC (83.78%) are slightly lower compared to the baseline Logistic Regression model trained on the full feature set.
This indicates that the removed features may contain valuable information for prediction.

**Comparison to Baseline:**
Baseline Accuracy: 80.01% (full feature set).
Selected Features Accuracy: 79.66%.

The minimal performance difference suggests that feature selection is unnecessary for this dataset when using Logistic Regression.

### 5.2 Decision Tree Model with Hyperparameter Tuning
This step optimizes the Decision Tree Classifier (DT) by tuning its hyperparameters to find the best configuration for maximizing model performance. The tuned model is evaluated on the test data using standard metrics.

In [ ]:
# Define hyperparameter grid for tuning
param_grid = {
    'max_depth': [80, 60, 40, 20, 10, 5],
    'criterion': ['gini', 'entropy', 'log_loss']
}

# Tune the Decision Tree model using cross-validation
dt = tune_model(
    DecisionTreeClassifier(random_state=17), 
    param_grid, 
    X_train, 
    y_train, 
    scoring='roc_auc'
)

# Compute predicted probabilities for Decision Tree
y_test_proba_dt = dt.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_pred = dt.predict(X_test)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_pred)  # Accuracy
f1 = f1_score(y_test, y_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_dt)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_dt, tpr_dt, _ = roc_curve(y_test, y_test_proba_dt)  # For ROC curve plotting later


#### Analysis of Desicion Tree
**DT Model Performance:**

The tuned Decision Tree model underperformed on the test set, with:
Accuracy of 72.68%. 
A significant drop in F1-Score (50.11%) and AUC-ROC (65.76%).
These results suggest that the Decision Tree struggled to generalize and may have lost critical information due to its limited depth.

**Comparison to Logistic Regression:**

Logistic Regression (with or without PCA) outperformed the Decision Tree in all metrics (accuracy, F1, AUC).
This indicates that Logistic Regression's simplicity and linear decision boundaries may be better suited for this dataset.

**Impact of Depth Restriction:**

While restricting the depth (max_depth=5) prevents overfitting, it may have been too aggressive, reducing the model's capacity to learn meaningful patterns.

**Conclusion**

The tuned Decision Tree model did not perform as well as Logistic Regression. This suggests that Decision Trees might not be the best standalone model for this dataset without further enhancements (e.g., using ensemble methods like Random Forest or Gradient Boosting).

### 5.3 Random Forest: Hyperparameter Tuning and Evaluation
To train and evaluate a Random Forest (RF) model by optimizing key hyperparameters using RandomizedSearchCV. The goal is to find the best configuration that maximizes performance on the test dataset.

In [ ]:
# Define hyperparameter grid for tuning
param_grid = {
    'criterion': ['gini', 'entropy'],         # Splitting criteria
    'n_estimators': [50, 100, 150],          # Number of trees in the forest
    'max_features': [40, 60, 80, 100],       # Maximum number of features to consider
    'max_depth': [80, 50, 20, 10, 5]         # Maximum depth of the trees
}

# Tune the Random Forest model using cross-validation
rf = tune_model(
    RandomForestClassifier(random_state=17, n_jobs=-1), 
    param_grid, 
    X_train, 
    y_train, 
    scoring='roc_auc'
)

# Compute predicted probabilities for Random Forest
y_test_proba_rf = rf.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_pred = rf.predict(X_test)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_pred)  # Accuracy
f1 = f1_score(y_test, y_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_rf)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_test_proba_rf)  # For ROC curve plotting later


#### Analysis of Random Forest
**Model Performance:** Random Forest achieved moderate test accuracy (79.18%) and AUC (81.93%) but did not outperform Logistic Regression. The F1-Score (57.28%) reflects limited handling of class imbalance.

**Comparison to Decision Tree:** Random Forest slightly improved accuracy and AUC over the Decision Tree, demonstrating better generalization and reduced overfitting.

**Depth Restriction:** The best max_depth=5 likely limited the model's ability to capture complex patterns.

**Feature Subsets:** Using max_features=40 helped reduce noise but may have restricted the model's full potential.

**Conclusion:**
Random Forest showed slight improvements over the Decision Tree but fell short of Logistic Regression. Increasing max_depth and max_features could improve performance.

### 5.4 Support Vector Machine (SVM): Hyperparameter Tuning and Evaluation
To optimize and evaluate an SVM classifier by tuning hyperparameters such as C, gamma, and kernel using cross-validation. The goal is to achieve high performance on the test dataset, focusing on the AUC metric.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve

# Define hyperparameter grid for tuning
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
param_grid = [
    {'C': param_range, 'kernel': ['linear']},
    {'C': param_range, 'gamma': param_range, 'kernel': ['rbf']}
]

# SVC model initialization with `probability=True`
svm = tune_model(
    SVC(probability=True, random_state=17), 
    param_grid, 
    X_train, 
    y_train, 
    scoring='roc_auc'
)

# Compute predicted probabilities for SVM
y_test_proba_svm = svm.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_pred = svm.predict(X_test)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_pred)  # Accuracy
f1 = f1_score(y_test, y_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_svm)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_svm, tpr_svm, _ = roc_curve(y_test, y_test_proba_svm)  # For ROC curve plotting later


#### Analysis of SVM Model
**Model Performance:**

The SVM model achieved moderate accuracy (79.60%) and AUC (78.71%) but struggled with balancing precision and recall, as reflected by the F1-Score (55.02%).
This indicates that the SVM had difficulty handling the class imbalance.

**Hyperparameter Impact:**

The use of the RBF kernel suggests that a non-linear decision boundary was more effective than a linear one.
A low C value (0.1) implies that stronger regularization was beneficial, likely preventing overfitting.

**Comparison to Other Models:**

The SVM's performance was similar to Random Forest and Decision Tree but did not outperform Logistic Regression in any metric.

**Conclusion:**
SVM with the RBF kernel provided reasonable results but did not outperform simpler models like Logistic Regression. Its sensitivity to hyperparameters and the need for scaling make it less favorable for this dataset.

### 5.5 K-Nearest Neighbors (KNN): Hyperparameter Tuning and Evaluation
To optimize the K-Nearest Neighbors (KNN) model by tuning the n_neighbors hyperparameter to identify the optimal number of neighbors. The goal is to achieve the best performance in terms of AUC and other metrics on the test dataset.

In [ ]:
import warnings
warnings.filterwarnings("ignore")  # Suppress warnings for cleaner output

# Define hyperparameter grid for tuning
param_grid = {'n_neighbors': [4, 6, 8, 10, 12]}

# Tune the KNN model using cross-validation
knn = tune_model(
    KNeighborsClassifier(), 
    param_grid, 
    X_train, 
    y_train, 
    scoring='roc_auc'
)

# Compute predicted probabilities for KNN
y_test_proba_knn = knn.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_pred = knn.predict(X_test)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_pred)  # Accuracy
f1 = f1_score(y_test, y_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_knn)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_test_proba_knn)  # For ROC curve plotting later


#### Analysis of KNN
**Model Performance:**

The KNN model achieved moderate test accuracy (75.52%) and AUC (76.13%), but the F1-Score (53.48%) highlights challenges in handling class imbalance.

**Hyperparameter Impact:**

The optimal n_neighbors=12 suggests that considering a larger number of neighbors helped smooth out predictions but may have diluted the impact of closer neighbors, limiting the model's ability to capture fine-grained patterns.

**Comparison to Other Models:**

KNN’s accuracy and AUC are lower than Logistic Regression, Random Forest, and SVM, indicating it is less effective for this dataset.
Its sensitivity to feature scaling and distance-based calculations may contribute to its lower performance.

**Conclusion:**
The KNN model provided lower performance compared to other models, with limited ability to balance class predictions effectively. Its reliance on the choice of neighbors and sensitivity to scaling makes it less suitable for this dataset.

### 5.6 Voting Classifier: Model Combination and Evaluation
To combine multiple individual classifiers into a Voting Classifier and evaluate its performance. The aim is to leverage the strengths of each model by combining their predictions in a "soft" voting mechanism to improve overall accuracy and robustness.

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve

# Define individual classifiers
clf1 = LogisticRegression(solver='liblinear', C=10.0, random_state=17)             # Logistic Regression
clf2 = DecisionTreeClassifier(max_depth=5, criterion='entropy', random_state=17)   # Decision Tree
clf3 = SVC(probability=True, kernel='rbf', gamma=0.01, C=0.1, random_state=17)     # Support Vector Machine
clf4 = KNeighborsClassifier(n_neighbors=4)                                         # K-Nearest Neighbors

# Define the Voting Classifier with "soft" voting
voting_clf = VotingClassifier(
    estimators=[('lr', clf1), ('dt', clf2), ('svm', clf3), ('knn', clf4)],
    voting='soft'
)

# Train the Voting Classifier on the training data
voting_clf.fit(X_train, y_train)

# Compute predicted probabilities for the test set
y_test_proba_voting_clf = voting_clf.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_pred = voting_clf.predict(X_test)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_pred)  # Accuracy
f1 = f1_score(y_test, y_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_voting_clf)

# Print evaluation metrics
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_voting_clf, tpr_voting_clf, _ = roc_curve(y_test, y_test_proba_voting_clf)  # For ROC curve plotting later


#### Analysis of Voting Classifier
**Model Combination:**

The Voting Classifier combines predictions from Logistic Regression, Decision Tree, SVM, and KNN using "soft" voting, which averages probabilities from each model to make predictions.
The ensemble approach helps leverage the strengths of each model but does not significantly outperform the best individual model (e.g., Logistic Regression).

**Performance Comparison:**
The Voting Classifier’s accuracy (79.54%) and AUC (82.84%) are similar to those of Logistic Regression, indicating limited added value from the ensemble.

**Class Imbalance:**
The moderate F1-Score (59.00%) suggests that the Voting Classifier struggles with class imbalance, similar to individual models.

**Conclusion:**
The Voting Classifier provided comparable performance to the best individual models but did not significantly improve overall metrics. This suggests limited synergy between the chosen classifiers.

### 5.7 Bagging Classifier: Implementation and Evaluation
To evaluate the performance of a Bagging Classifier, an ensemble method that trains multiple instances of a base classifier (in this case, a Decision Tree) on different subsets of the data. The goal is to improve model stability and reduce variance compared to a single Decision Tree.

In [ ]:
# Define the base estimator: Decision Tree
tree = DecisionTreeClassifier(
    criterion='entropy', 
    max_depth=None, 
    random_state=17
)

# Define the Bagging Classifier
bag = BaggingClassifier(
    estimator=tree,               # Base classifier
    n_estimators=500,             # Number of base classifiers
    max_samples=1.0,              # Use all samples (with replacement)
    max_features=0.4,             # Use 40% of features for each base estimator
    bootstrap=True,               # Bootstrap samples
    bootstrap_features=False,     # Do not bootstrap features
    n_jobs=-1,                    # Use all processors for parallel computation
    random_state=17
)

# Train the Bagging Classifier
bag = bag.fit(X_train, y_train)

# Compute predicted probabilities for the Bagging Classifier
y_test_proba_bag = bag.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_test_pred = bag.predict(X_test)

# Evaluate the model using hard predictions
accuracy = accuracy_score(y_test, y_test_pred)  # Accuracy
f1 = f1_score(y_test, y_test_pred)              # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_bag)

# Print evaluation metrics
print(f'Bagging Train Accuracy: {accuracy_score(y_train, bag.predict(X_train)):.3f}')
print(f'Bagging Test Accuracy: {accuracy:.3f}')
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_bag, tpr_bag, _ = roc_curve(y_test, y_test_proba_bag)  # For ROC curve plotting later


#### Analysis of Bagging Classifier
**Training Accuracy (99.0%):** Indicates overfitting due to unpruned Decision Trees (max_depth=None), which memorize the training data.

**Test Accuracy (79.0%):** Suggests moderate generalization but highlights a significant drop from training accuracy.

**ROC-AUC (82.27%):** Demonstrates good ranking ability, outperforming test accuracy, and indicating robust performance in distinguishing classes.

**Conclusion:**
The Bagging Classifier performs reasonably well on the test data with a strong ROC-AUC but suffers from overfitting, as shown by the training-test accuracy gap. Further tuning is recommended.

### 5.8 XGBoost Classifier: Implementation and Evaluation
To evaluate the performance of the XGBoost Classifier, a powerful gradient boosting algorithm, by training it on the dataset and calculating relevant metrics, including accuracy and ROC-AUC.



In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, f1_score
import xgboost as xgb

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(
    n_estimators=500,        # Number of boosting rounds
    learning_rate=0.01,      # Step size shrinkage to prevent overfitting
    max_depth=4,             # Maximum tree depth for base learners
    random_state=17          # Ensure reproducibility
)

# Train the XGBoost Classifier
xgb_clf.fit(X_train, y_train)

# Compute predicted probabilities for XGBoost
y_test_proba_xgb = xgb_clf.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Make hard predictions on the test set
y_test_pred = xgb_clf.predict(X_test)

# Evaluate the model using hard predictions
xgb_train_accuracy = accuracy_score(y_train, xgb_clf.predict(X_train))  # Training Accuracy
xgb_test_accuracy = accuracy_score(y_test, y_test_pred)                 # Test Accuracy
f1 = f1_score(y_test, y_test_pred)                                      # F1 Score

# Evaluate the ROC-AUC using probabilities
roc_auc = roc_auc_score(y_test, y_test_proba_xgb)

# Print evaluation metrics
print(f'XGBoost Train Accuracy: {xgb_train_accuracy:.3f}')
print(f'XGBoost Test Accuracy: {xgb_test_accuracy:.3f}')
print(f"Test F1: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Prepare values for later plotting (store in variables or dictionary)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_test_proba_xgb)  # For ROC curve plotting later


#### Analysis
**Model Performance:** Training accuracy (82.8%) and test accuracy (79.2%) are close, indicating minimal overfitting and good generalization. The ROC-AUC (83.74%) shows strong class distinction.

**Strengths of XGBoost:** Outperforms standalone models (e.g., Decision Tree, KNN) by reducing bias and variance. Gradual boosting with a learning rate of 0.01 and 500 estimators enhances stability.

**Comparison:** Achieves the highest ROC-AUC among tested models, highlighting its ability to rank predictions effectively.

**Conclusion:**
XGBoost delivers strong performance, balancing generalization and ranking ability better than previous models.

### 5.9 XGBoost Classifier: Hyperparameter Tuning and Evaluation
To improve the XGBoost Classifier's performance by using RandomizedSearchCV to find the best hyperparameters and evaluate its performance on the test set. Metrics include accuracy, F1 score, ROC-AUC, and an optimized F1 score based on threshold adjustment.

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Define updated parameter grid
param_grid = {
    'n_estimators': [300, 500, 700],        # Number of boosting rounds
    'learning_rate': [0.01, 0.02, 0.05],   # Learning rate for fine updates
    'max_depth': [4, 5, 6],                # Allow more complex trees
    'min_child_weight': [1, 2],            # Looser constraints on leaf nodes
    'subsample': [0.8, 0.9, 1.0],          # Higher data usage
    'colsample_bytree': [0.8, 0.9, 1.0],   # Higher feature usage
    'gamma': [0, 0.1, 0.2],                # Regularization for tree splits
    'reg_alpha': [0.0, 0.1],               # Reduced L1 regularization
    'reg_lambda': [0.5, 1.0, 1.5],         # Reduced L2 regularization
}

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(
    random_state=17,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_grid,
    n_iter=50,              # Number of settings to sample
    scoring='roc_auc',      # Optimize for ROC-AUC
    cv=5,                   # Stratified k-fold cross-validation
    verbose=0,
    random_state=42,
    n_jobs=-1               # Use all CPU cores
)

# Fit the model with RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best model and parameters
best_xgb = random_search.best_estimator_
print(f"Best Parameters: {random_search.best_params_}")

# Evaluate on the test set
y_test_proba_best_xgb = best_xgb.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
y_test_pred = best_xgb.predict(X_test)

# Performance metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_proba_best_xgb)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test ROC-AUC: {test_auc:.4f}")

# Threshold Optimization
precisions, recalls, thresholds = precision_recall_curve(y_test, y_test_proba_best_xgb)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
optimal_threshold_idx = f1_scores.argmax()
optimal_threshold = thresholds[optimal_threshold_idx]  # Get the optimal threshold
print(f"Optimal Threshold: {optimal_threshold:.4f}")

# Reevaluate with the optimal threshold
y_test_pred_opt = (y_test_proba_best_xgb >= optimal_threshold).astype(int)
test_f1_opt = f1_score(y_test, y_test_pred_opt)
print(f"Optimized Test F1 Score: {test_f1_opt:.4f}")


### Key Improvements:
**Threshold Optimization:** F1 score increased from 0.5473 to 0.6389, balancing precision and recall effectively.

**ROC-AUC Stability:** Both models maintained strong ROC-AUC scores (0.8374 and 0.8372), ensuring reliability.

The tuned model includes threshold optimization, which improves its F1 score to 0.6389. This gives it a clear advantage when precision-recall balance is critical.

## 6. Code Implementation Comparison
### 6.1 Performance Metrics Table

In [ ]:
import pandas as pd

# Define the metrics dictionary
model_metrics = {
    "Logistic Regression (Baseline)": {"Accuracy": 0.8001, "F1-Score": 0.6024, "ROC-AUC": 0.8383},
    "Logistic Regression (Feature Selection)": {"Accuracy": 0.7966, "F1-Score": 0.5934, "ROC-AUC": 0.8378},
    "Decision Tree": {"Accuracy": 0.7268, "F1-Score": 0.5011, "ROC-AUC": 0.6576},
    "Random Forest": {"Accuracy": 0.7918, "F1-Score": 0.5728, "ROC-AUC": 0.8193},
    "SVM": {"Accuracy": 0.7960, "F1-Score": 0.5502, "ROC-AUC": 0.7871},
    "KNN": {"Accuracy": 0.7552, "F1-Score": 0.5348, "ROC-AUC": 0.7613},
    "Voting Classifier": {"Accuracy": 0.7954, "F1-Score": 0.5884, "ROC-AUC": 0.8284},
    "Bagging Classifier": {"Accuracy": 0.7900, "F1-Score": 0.5298, "ROC-AUC": 0.8227},
    "XGBoost (Tunned) ": {"Accuracy": 0.7847, "F1-Score": 0.6389, "ROC-AUC": 0.8372},
}

# Convert metrics dictionary to a DataFrame
metrics_df = pd.DataFrame(model_metrics).T  # Transpose to make models the rows
metrics_df.index.name = "Model"  # Add a name to the index
metrics_df.reset_index(inplace=True)  # Move the index to a regular column

# Round metrics to 4 decimal places
metrics_df = metrics_df.round(4)

# Print the table with a better format
print(metrics_df.to_markdown(index=False))


### 6.2 Visualize Metrics
Grouped Bar Chart for Accuracy, F1-Score, and ROC-AUC

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert metrics to DataFrame for plotting
metrics_df = pd.DataFrame(model_metrics).T  # Transpose to have models as rows
metrics_df.reset_index(inplace=True)        # Reset index for plotting
metrics_df.rename(columns={"index": "Model"}, inplace=True)

# Plot grouped bar chart
metrics_df.plot(x="Model", kind="bar", figsize=(14, 8), legend=True)
plt.title("Comparison of Model Metrics")
plt.ylabel("Scores")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Metrics")
plt.tight_layout()
plt.show()


### 6.3 Plotting the ROC Curves
This will compare the ranking ability of all models. The ROC (Receiver Operating Characteristic) curve compares the True Positive Rate (TPR) against the False Positive Rate (FPR) for different classification thresholds. The Area Under the Curve (AUC) quantifies the model's ability to distinguish between positive and negative classes.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Ensure model_metrics is populated correctly
model_metrics = {
    "Logistic Regression (Baseline)": {"ROC-AUC": roc_auc_score(y_test, y_test_proba_lr_baseline)},
    "Decision Tree": {"ROC-AUC": roc_auc_score(y_test, y_test_proba_dt)},
    "Random Forest": {"ROC-AUC": roc_auc_score(y_test, y_test_proba_rf)},
    "KNN": {"ROC-AUC": roc_auc_score(y_test, y_test_proba_knn)},
    "SVM": {"ROC-AUC": roc_auc_score(y_test, y_test_proba_svm)},
    "Voting Classifier": {"ROC-AUC": roc_auc_score(y_test, y_test_proba_voting_clf)},
    "Bagging Classifier": {"ROC-AUC": roc_auc_score(y_test, y_test_proba_bag)},
    "XGBoost": {"ROC-AUC": roc_auc_score(y_test, y_test_proba_xgb)},
}

# Plotting ROC Curves
plt.figure(figsize=(10, 8))

# Define model probabilities and labels for plotting
models = {
    "Logistic Regression (Baseline)": y_test_proba_lr_baseline,
    "Decision Tree": y_test_proba_dt,
    "Random Forest": y_test_proba_rf,
    "KNN": y_test_proba_knn,
    "SVM": y_test_proba_svm,
    "Voting Classifier": y_test_proba_voting_clf,
    "Bagging Classifier": y_test_proba_bag,
    "XGBoost": y_test_proba_xgb,
}

# Iterate over models and plot their ROC curves
for model_name, y_proba in models.items():
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc_score = model_metrics[model_name]["ROC-AUC"]
    plt.plot(fpr, tpr, label=f"{model_name} (AUC = {auc_score:.2f})")

# Plot configuration
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random Chance")
plt.title("ROC Curves for All Models")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


### ROC Curve Results:

**XGBoost (AUC = 0.84):**

XGBoost demonstrates the best performance among all models, achieving the highest AUC. Its curve closely follows the top-left corner, signifying strong discriminatory power and an excellent balance between sensitivity (True Positive Rate) and specificity (False Positive Rate). XGBoost remains the most robust candidate for the final model due to its high AUC and superior ability to handle complex patterns in the data.

**Logistic Regression (AUC = 0.84)**

Logistic Regression shows excellent performance, matching XGBoost in terms of AUC. This is the second-best performer. Its simplicity and interpretability make it an attractive option. Despite being a baseline model, its performance demonstrates that linear models can be highly effective in certain contexts. 

**Voting Classifier (AUC = 0.83):**

The Voting Classifier also performs very well. The performance is slightly below XGBoost and LR but still competitive.
This suggests that combining multiple models (Logistic Regression, Decision Tree, SVM, KNN) has synergistic benefits, improving the ability to rank predictions.

**Bagging Classifier (AUC = 0.82):**

Bagging provides strong performance, with an AUC comparable to the Voting Classifier. Its curve is smooth and indicates robust classification across thresholds. However, it falls slightly behind XGBoost, Logistic Regression and the Voting Classifier in ranking ability. Bagging remains a solid choice if simplicity and robustness are prioritized.

**Random Forest (AUC = 0.82)** 

Random Forest matches the AUC of Bagging, showcasing its ability to rank predictions effectively. However, it does not outperform XGBoost or the Voting Classifier, indicating that boosting methods and other ensemble strategies might handle the dataset complexity better.

**KNN (AUC = 0.76) and SVM (AUC = 0.79)**

Both KNN and SVM show moderate performance but underperform compared to ensemble methods and Logistic Regression. Their curves indicate less effective handling of class imbalance, with flatter ROC curves suggesting weaker ranking ability.

**Decision Tree (AUC = 0.66)**

The Decision Tree is the weakest model, with the lowest AUC and the most distant curve from the top-left corner. Its performance indicates significant limitations in handling the dataset's complexity and distinguishing between classes effectively.

**Conclusion:**

Both **XGBoost** and **Logistic Regression** are the strongest candidates for the final model, as they achieve the same **AUC (0.84)** and very similar scores.

The **Voting Classifier** is a strong alternative if an ensemble approach is desired, while **Bagging** and **Random Forest** provide solid, but not top-tier, performance. Individual classifiers like **KNN, SVM,** and **Decision Tree** are less effective in this task.

## 7. Submission
### 7.1 Logistic Regression (Baseline) - score 85.625%

In [ ]:
# Generate submission file using the LR model
model=lr_baseline
write_predictions(model, test_input_path='/kaggle/input/customer-churn-prediction-fall-2024/test.csv', 
                      predict_out_path='lr_submission.csv')

### 7.2 XBoost Tunned - score 85.634%

In [ ]:
# Generate submission file
write_predictions(
    model=best_xgb, 
    test_input_path='/kaggle/input/customer-churn-prediction-fall-2024/test.csv', 
    predict_out_path='best_xgb_submission.csv'  # Updated file name for clarity
)


## Final Conclusion:

While **Logistic Regression** performed exceptionally well, achieving an score of **85.625%**, we ultimately finalized **XGBoost**, which achieved a slightly better score of **85.634%** after tuning. This marginal improvement might seem small, but there are three key reasons for our decision:

**1) Threshold Optimization:** We tuned XGBoost with an optimal threshold, improving its F1-Score to 0.6389 compared to Logistic Regression's 0.6024. This makes XGBoost superior when balancing precision and recall is critical—an essential factor in churn prediction.

**2. Flexibility and Potential:** Feature selection and PCA did not improve Logistic Regression, suggesting limited room for optimization. In contrast, XGBoost provided opportunities for fine-tuning, demonstrating its ability to adapt to the dataset.

**3. Probability-Based Decisions:** Both models provide probabilities, but XGBoost’s enhanced ranking ability and flexibility ensure it’s better equipped for prioritizing customers with high churn probabilities.

Additionally, **Voting Classifier, Bagging, and Random Forest** all achieved similar **AUC scores (0.82–0.83)**, demonstrating good performance and effective ranking abilities. However, they slightly lagged behind XGBoost and Logistic Regression in handling the dataset's complexity and optimizing precision-recall trade-offs.

**In conclusion,** while Logistic Regression offered simplicity and competitive performance, **XGBoost emerged as the stronger candidate** for predicting churn. Its threshold optimization, improved F1-Score, and robust ranking ability give it a clear edge. By leveraging the predicted probabilities, XGBoost empowers the company to take precise, data-driven actions to retain high-risk customers.